In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('./data/mixed_first_marketplace.sqlite3')

cursor = conn.cursor()

# cursor.execute("SELECT feedbacks.order_amount_usd, items.total_sales FROM feedbacks, items WHERE items.hash_str = feedbacks.item_hash")

# for each item with unique item_hash
# we have the total_sales
# for each feedback, we have the transaction amount
# sum all the transaction amount from the feedbacks with the same item_hash
# divide the total sales, this should give per-item-price

cursor.execute("SELECT item_hash, SUM(order_amount_usd) AS total_order_amount \
               FROM feedbacks \
               WHERE (marketplace = 'Agora' OR marketplace = 'Evolution') \
               GROUP BY item_hash")

result = cursor.fetchall()

column_names = [description[0] for description in cursor.description]
dff = pd.DataFrame(result, columns=column_names)
dff.to_csv("./data/total_order_amount_by_item.csv", index=False)

cursor.close()
conn.close()


In [19]:
import pandas as pd
import csv

df = pd.read_csv("./data/mixed_first_items_merged.csv")
column_names = df.columns
print(column_names)

columns_to_keep = ['marketplace', 'item_id', 'date', 'sales', 'vendor_name', 'author', 'ships_from_coded', 'ships_to', 'category', 'if_drug', 'top_category', '2nd_level_category']

df = df[columns_to_keep]

df = df[df['marketplace'].isin(['agora', 'evolution'])]

print(df)

df1 = pd.read_csv("./data/total_order_amount_by_item.csv")

merged_df = df.merge(df1, left_on='item_id', right_on='item_hash', how='left')

# Calculate the 'price' column
merged_df['price'] = merged_df['total_order_amount'] / merged_df['sales']

merged_df.to_csv("./data/new_output.csv", index=False)



Index(['review_id', 'marketplace', 'item_id', 'date', 'sales',
       'feedback_value', 'vendor_name', 'author', 'ships_from_coded',
       'ships_to', 'category', 'if_drug', 'top_category',
       '2nd_level_category'],
      dtype='object')
       marketplace                           item_id        date   sales  \
0            agora  b03a8d9cfc1f7bf82222149435845a52  2013-12-22   64.06   
1            agora  e6a55e17458c00d0c5084855fe99c099  2013-12-22   89.02   
2            agora  879c7446c8327f761ad2f2a9e31d3b38  2013-12-23  763.05   
3            agora  c000e49b4639d9701759d54c0bb434e5  2013-12-24   67.08   
4            agora  e88ff0ac1cf19e7a09d1a8679218e916  2013-12-24  152.61   
...            ...                               ...         ...     ...   
698513   evolution  e9bdbee66bd141df3762146093970f14  2015-02-19    0.00   
698514   evolution  da01cfdd88c4862c3de26563f6c9be2f  2015-02-19    0.00   
698515   evolution  ea9c2cc525f7f90c9772d902ca56a18c  2015-02-19    0.00 

In [ ]:
df = pd.read_csv("./data/new_output.csv")

agora_df = df[df['marketplace'].isin(['agora'])]
evolution_df = df[df['marketplace'].isin(['evolution'])]

agora_df.to_csv("./data/Agora_new_output.csv", index=False)
evolution_df.to_csv("./data/Evolution_new_output.csv", index=False)

In [12]:
merged_df

,marketplace,item_id,date,sales,vendor_name,author,ships_from_coded,ships_to,category,if_drug,top_category,2nd_level_category,item_hash,total_order_amount,price


In [ ]:
['marketplace', 'item_id', 'date', 'sales', 'price', 'vendor_name', 'author', 
 'ships_from_coded', 'ships_to', 'category', 'if_drug', 'top_category', '2nd_level_category']

# marketplace -> name of the market
# item id -> mixed_first_items_merged.csv : item_id
# date -> mixed_first_items_merged.csv : date
# sales -> mixed_first_items_merged.csv : sales
# price -> sqlite3 : feedbacks : order
# vendor_name -> mixed_first_items_merged.csv : 
# author -> mixed_first_items_merged.csv : 
# ships_from_coded -> mixed_first_items_merged.csv : 
# ships_to -> mixed_first_items_merged.csv : 
# category -> mixed_first_items_merged.csv : 
# if_drug -> mixed_first_items_merged.csv : 
# top_category -> mixed_first_items_merged.csv : 
# 2nd_level_category -> mixed_first_items_merged.csv : 